<a href="https://colab.research.google.com/github/ShaswataJash/kfpcomponent/blob/main/TabularDataVisualizationUsingPandaProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the development workflow for kubeflow pipeline component of the same name as this notebook. Refer https://github.com/ShaswataJash/kfpcomponent

#Install required softwares

In [1]:
!uname -a

Linux e751bd1d8376 5.4.188+ #1 SMP Sun Apr 24 10:03:06 PDT 2022 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [3]:
!python --version

Python 3.7.13


In [4]:
!pip install pandas-profiling==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 262 kB 5.1 MB/s 
     |████████████████████████████████| 11.1 MB 24.9 MB/s 
     |████████████████████████████████| 102 kB 7.7 MB/s 
     |████████████████████████████████| 4.7 MB 38.5 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 690 kB 48.6 MB/s 
     |████████████████████████████████| 812 kB 54.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=d846de11057b142347a139a8928b1cd56d02107d11d170fc8dbea29d45a7d273
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=224de9754ec7b72a75b70d9351

In [5]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [20]:
#needed for test validation
!pip install beautifulsoup4==4.11.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [6]:
!apt-get install ca-certificates fuse tzdata curl unzip && \
  echo "user_allow_other" >> /etc/fuse.conf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fuse is already the newest version (2.9.7-1ubuntu1).
ca-certificates is already the newest version (20210119~18.04.2).
curl is already the newest version (7.58.0-2ubuntu3.18).
tzdata is already the newest version (2022a-0ubuntu0.18.04).
tzdata set to manually installed.
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [7]:
!curl https://rclone.org/install.sh | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4497  100  4497    0     0   5714      0 --:--:-- --:--:-- --:--:--  5706
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/cs...
Processing manual pages under /usr/share/man/cs...
Purging old database entries

In [8]:
!rclone --version

rclone v1.58.1
- os/version: ubuntu 18.04 (64 bit)
- os/kernel: 5.4.188+ (x86_64)
- os/type: linux
- os/arch: amd64
- go/version: go1.17.9
- go/linking: static
- go/tags: none


#Develop source code files

In [55]:
%%writefile data_visualization.py
#!/usr/bin/env python3
import argparse

# Defining and parsing the command-line arguments
parser = argparse.ArgumentParser(description='kubeflow pipeline component to read csv file and visualize the data')
parser.add_argument('--input-datasource-directory-mountable', default=False, action="store_true", help='whether input csv file is present in mountable remote location')
parser.add_argument('--input-datasource-directory-to-be-mounted', type=str, help='if input-datasource-directory-mountable=True, name of the mountable directory (e.g. bucket name for s3)')
parser.add_argument('--input-datasource-file-name', type=str, help='name of the csv file including file extension (if any)')
parser.add_argument('--additional-options-csv-parsing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.read_csv()')
parser.add_argument('--panda-profiler-options', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to ProfileReport()')
parser.add_argument('--sensitive-data-present', default=False, action="store_true", help='whether some sensitive data present (if so, panda-profiler will not show sample data etc.)')
parser.add_argument('--output-visualization-absolute-path', type=str, help='html filepath having visualization report')
args = parser.parse_args()

import tempfile
local_datastore_read_dir = tempfile.mkdtemp(prefix="my_local_read-")
print('local_datastore_read_dir:',local_datastore_read_dir)

#input file handling
import subprocess
import os
import sys
if args.input_datasource_directory_mountable:
    input_data_read_cmd = "rclone -v mount remoteread:" + args.input_datasource_directory_to_be_mounted + ' ' + local_datastore_read_dir + ' --daemon'
else:
    input_data_read_cmd = "rclone -v copy remoteread:" + args.input_datasource_file_name + ' ' + local_datastore_read_dir
input_data_read_call = subprocess.run(input_data_read_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(input_data_read_call.stdout)
if input_data_read_call.returncode != 0:
    print("Error in rclone, errorcode=", input_data_read_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of reading")

#handling input csv file reading
import pandas
import json

try:
    parse_config = json.loads(args.additional_options_csv_parsing)
    print('parse_config = (', type(parse_config), ')', parse_config)
    parse_config['filepath_or_buffer'] = os.path.join(local_datastore_read_dir,args.input_datasource_file_name)
    my_data = pandas.read_csv(**parse_config)
    print(my_data)
    
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while parsing input csv file")

#generating output html report
from pandas_profiling import ProfileReport
try: 
    profile_config = json.loads(args.panda_profiler_options)
    print('profile_config = (', type(profile_config), ')', profile_config)
    profile_config['progress_bar'] = False
    profile = ProfileReport(my_data, **profile_config)
    profile.to_file(args.output_visualization_absolute_path)
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while profiling the data")

#dumping kubeflow pipeline compatible meta data for visualization
#refer: https://www.kubeflow.org/docs/components/pipelines/sdk/output-viewer/#web-app

with open(args.output_visualization_absolute_path, 'r') as html_rep:
    html_content = html_rep.read()

metadata = {
    'outputs' : [{
      'type': 'web-app',
      'storage': 'inline',
      'source': html_content,
    }]
}
with open('/tmp/mlpipeline-ui-metadata.json', 'w') as f:
    json.dump(metadata, f)

Overwriting data_visualization.py


Docker size reduction tips:


*   https://devopscube.com/reduce-docker-image-size/
*   https://www.ecloudcontrol.com/best-practices-to-reduce-docker-images-size/



In [56]:
%%writefile Dockerfile
FROM python:3.7.13-slim

RUN python3 -m pip install pandas-profiling==3.2.0
RUN python3 -m pip install markupsafe==2.0.1
RUN python3 -m pip install beautifulsoup4==4.11.1

#install fuse as dependency for rclone. Additionally, install curl, unzip for rclone installer to work
RUN apt-get update \
    && apt-get install --no-install-recommends -y curl fuse unzip \
    && echo "user_allow_other" >> /etc/fuse.conf \
    && curl https://rclone.org/install.sh | bash \
    && apt-get -y remove --purge curl unzip \
    && rm -rf /var/lib/apt/lists/* \
    && rclone --version

COPY src/data_visualization.py /tmp
COPY tests/test_validation.py /tmp
COPY run_tests.sh /tmp
RUN chmod 544 /tmp/run_tests.sh

Writing Dockerfile


In [57]:
%%writefile run_tests.sh
#!/bin/bash

export RCLONE_CONFIG_REMOTEREAD_TYPE='http'
#Dataset description: https://archive.ics.uci.edu/ml/datasets/adult
export RCLONE_CONFIG_REMOTEREAD_URL='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/'

mkdir /tmp/my_local_dir_for_test

#refer: https://github.com/ydataai/pandas-profiling/blob/master/src/pandas_profiling/config_default.yaml
#override panda-profiler options on top of config_default in --panda-profiler-options below.
#Test: visualization of csv file, rclone read in copy mode
python /tmp/data_visualization.py --input-datasource-file-name 'adult.data' \
    --additional-options-csv-parsing '{"names":["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income-class"]}' \
    --panda-profiler-options '{"title":"US Adult Census Income prediction", "memory_deep":true, "html":{"full_width": true}}' \
    --output-visualization-absolute-path '/tmp/my_local_dir_for_test/adult_data_report.html'

python /tmp/test_validation.py

Overwriting run_tests.sh


In [58]:
%%writefile test_validation.py
#!/usr/bin/env python3
from bs4 import BeautifulSoup
with open('/tmp/my_local_dir_for_test/adult_data_report.html') as fp:
    soup = BeautifulSoup(fp, "html.parser")
assert soup.title.get_text() == 'US Adult Census Income prediction'

table = soup.find('table', attrs={'class':'dataframe duplicate table table-striped'})

import pandas as pd
duplicated_row_info_df = pd.read_html(str(table))[0]
assert len(duplicated_row_info_df.index) == 10 #there are 10 uniquely identifyable duplicated rows
print("Test validation is done")

Overwriting test_validation.py


In [59]:
%%writefile docker-compose.test.yml
services:
  sut:
    build: .
    command: /tmp/run_tests.sh

Writing docker-compose.test.yml



*   https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#designing-a-pipeline-component
*   https://github.com/kubeflow/pipelines/blob/sdk/release-1.8/sdk/python/kfp/dsl/types.py



In [60]:
%%writefile component.yaml
name: TabularDataVisualizationUsingPandaProfiler
description: |
    Visualize tabular data (csv file) using pandas-profiler library. (Refer: https://github.com/ydataai/pandas-profiling)
    Refer panda-profiler-options in command line arguments. 
    pandas-profiler internally uses pandas dataframe to read csv file. You can utilize options exposed by panda's read_csv(). 
    Refer additional-options-csv-parsing in command line arguments
    Input csv file are stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Thus, before using this component in kubeflow pipeline, those environment variables have to be set from pipeline.
    Create rclone read configuration file name as 'REMOTEREAD'. Because the same is used within code.
    So convention for creating any environment variables related to rclone should start with 'RCLONE_CONFIG_REMOTEREAD'.
    Visualization data is emitted as self-sufficient single html file. 

inputs:
- {name: input-datasource-directory-mountable, optional, description: 'whether input csv file is present in mountable remote location'}
- {name: input-datasource-directory-to-be-mounted, type: String, description: 'if input-datasource-directory-mountable=True, name of the mountable directory (e.g. bucket name for s3)'}
- {name: input-datasource-file-name, type: String, description: 'name of the csv file including file extension (if any)'}
- {name: additional-options-csv-parsing, type: String, default= '{}', description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'}
- {name: panda-profiler-options, type: String, default= '{}', description:'json formatted key-value pairs of strings which will be passed to ProfileReport()'}
- {name: sensitive-data-present, optional, description: 'whether some sensitive data present (if so, panda-profiler will not show sample data etc.)'}
- {name: output-visualization-absolute-path, type: String, description: 'html filepath having visualization report'}

outputs:
- {name: MLPipeline UI metadata, type: UI metadata}

implementation:
  container:
    image: hub.docker.com/shasjash/kfpcomponents/TabularDataVisualizationUsingPandaProfiler_devlatest
    # command is a list of strings (command-line arguments). 
    # The YAML language has two syntaxes for lists and you can use either of them. 
    # Here we use the "flow syntax" - comma-separated strings inside square brackets.
    command: [
      python3, 
      # Path of the program inside the container
      /tmp/data_visualization.py,
      --input-datasource-directory-mountable,
      {inputValue: input-datasource-directory-mountable},
      --input-datasource-directory-to-be-mounted, 
      {inputValue: input-datasource-directory-to-be-mounted},
      --input-datasource-file-name, 
      {inputValue: input-datasource-file-name},
      --additional-options-csv-parsing, 
      {inputValue: additional-options-csv-parsing},
      --panda-profiler-options, 
      {inputValue: panda-profiler-options},
      --sensitive-data-present, 
      {inputValue: sensitive-data-present},
      --output-visualization-absolute-path, 
      {inputValue: output-visualization-absolute-path},
    ]
    fileOutputs:
      MLPipeline UI metadata:  /tmp/mlpipeline-ui-metadata.json

Writing component.yaml


#Software testing

Now let us simulate testing what will be done by docker-hub infrastructure as part of auto-testing by using docker-compose.test.yml present in github source repository.

In [61]:
!rm -rf /tmp/my_local_dir_for_test
!chmod 544 run_tests.sh
!cp data_visualization.py /tmp
!cp test_validation.py /tmp
!./run_tests.sh

local_datastore_read_dir: /tmp/my_local_read-upk_dk56
2022/06/11 20:09:09 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
2022/06/11 20:09:10 INFO  : adult.data: Copied (new)
2022/06/11 20:09:10 INFO  : 
Transferred:   	    3.790 MiB / 3.790 MiB, 100%, 0 B/s, ETA -
Transferred:            1 / 1, 100%
Elapsed time:         1.1s


parse_config = ( <class 'dict'> ) {'names': ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-class']}
       age  ... income-class
0       39  ...        <=50K
1       50  ...        <=50K
2       38  ...        <=50K
3       53  ...        <=50K
4       28  ...        <=50K
...    ...  ...          ...
32556   27  ...        <=50K
32557   40  ...         >50K
32558   58  ...        <=50K
32559   22  ...        <=50K
32560   52  ...         >50K

[32561 rows x 15 columns]
profile

#Push the code to github

Before commiting code to github, install github client (gh) by following instruction mentioned in https://github.com/cli/cli/blob/trunk/docs/install_linux.md (Choose Debian, Ubuntu Linux way of installation) 

Use the colab's 'Terminal' icon present in left vertical pane to open linux terminal to type commands. Once 'gh' is installed, type **$gh auth login** (refer https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git) to follow onscreen prompts. For colab, use **Paste an authentication token** option. Personal tokens can be generated in https://github.com/settings/tokens

You can use Shift+Ctrl+v shortcut to paste any string in colab console

In [62]:
!pwd

/content


In [63]:
!rm -Rf kfpcomponent

In [64]:
!git clone https://github.com/ShaswataJash/kfpcomponent.git

Cloning into 'kfpcomponent'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 150 (delta 77), reused 99 (delta 42), pack-reused 0
Receiving objects: 100% (150/150), 59.73 KiB | 5.43 MiB/s, done.
Resolving deltas: 100% (77/77), done.


Follow directory structure according to https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#organizing-the-component-files

In [65]:
!mkdir kfpcomponent/TabularDataVisualizationUsingPandaProfiler
!mkdir kfpcomponent/TabularDataVisualizationUsingPandaProfiler/src
!mkdir kfpcomponent/TabularDataVisualizationUsingPandaProfiler/tests

In [66]:
#it will ensure file is coped in git repo only if file content is changed by checking checksum of file content
!rsync -c data_visualization.py kfpcomponent/TabularDataVisualizationUsingPandaProfiler/src
!rsync -c component.yaml kfpcomponent/TabularDataVisualizationUsingPandaProfiler/component.yaml
!rsync -c test_validation.py kfpcomponent/TabularDataVisualizationUsingPandaProfiler/tests
!rsync -c Dockerfile kfpcomponent/TabularDataVisualizationUsingPandaProfiler/
!rsync -c run_tests.sh kfpcomponent/TabularDataVisualizationUsingPandaProfiler/
!rsync -c docker-compose.test.yml kfpcomponent/TabularDataVisualizationUsingPandaProfiler/

In [67]:
%cd kfpcomponent

/content/kfpcomponent


In [68]:
!git add -A

In [69]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   TabularDataVisualizationUsingPandaProfiler/Dockerfile
	new file:   TabularDataVisualizationUsingPandaProfiler/component.yaml
	new file:   TabularDataVisualizationUsingPandaProfiler/docker-compose.test.yml
	new file:   TabularDataVisualizationUsingPandaProfiler/run_tests.sh
	new file:   TabularDataVisualizationUsingPandaProfiler/src/data_visualization.py
	new file:   TabularDataVisualizationUsingPandaProfiler/tests/test_validation.py



For git-user who has set their email visibility as private, git provides alternate email address to use in web-based Git operations, e.g., edits and merges. The alias email can be viewed in https://github.com/settings/emails

In [70]:
!git config --global user.email "29448766+ShaswataJash@users.noreply.github.com"

In [71]:
!git commit -a -m "first commit"

[main 1cef073] first commit
 6 files changed, 181 insertions(+)
 create mode 100644 TabularDataVisualizationUsingPandaProfiler/Dockerfile
 create mode 100644 TabularDataVisualizationUsingPandaProfiler/component.yaml
 create mode 100644 TabularDataVisualizationUsingPandaProfiler/docker-compose.test.yml
 create mode 100755 TabularDataVisualizationUsingPandaProfiler/run_tests.sh
 create mode 100644 TabularDataVisualizationUsingPandaProfiler/src/data_visualization.py
 create mode 100644 TabularDataVisualizationUsingPandaProfiler/tests/test_validation.py


In [72]:
!git push origin main

Counting objects: 11, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (11/11), 4.52 KiB | 4.52 MiB/s, done.
Total 11 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/ShaswataJash/kfpcomponent.git
   55daea2..1cef073  main -> main


In [73]:
%cd ..

/content
